<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

**Сшивание данных по URL и категориальным признакам, а так же разбивка на train и test**

In [1]:
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import pandas as pd
import polars as pl
import bisect

In [2]:
targets = pd.read_parquet('public_train.pqt') 
sumbit = pd.read_parquet('submit_2.pqt')

In [3]:
targets.head()

,age,is_male,user_id
350459,31.0,1,350459
188276,35.0,1,188276
99002,41.0,0,99002
155506,33.0,0,155506
213873,54.0,0,213873


In [4]:
targets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270000 entries, 350459 to 103148
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   age      269999 non-null  float64
 1   is_male  269958 non-null  object 
 2   user_id  270000 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 8.2+ MB


In [5]:
targets = targets[targets['is_male'] != 'NA']
targets = targets[targets['age'] != 'NA']

In [6]:
targets.is_male = targets.is_male.astype('Int32')
targets.user_id = targets.user_id.astype('Int32')

In [7]:
targets = targets.dropna()

In [8]:
targets.age.unique()

array([31., 35., 41., 33., 54., 63., 39., 27., 66., 48., 60., 21., 47.,
       36., 49., 40., 57., 53., 51., 25., 59., 29., 55., 26., 42., 24.,
       64., 50., 45., 32., 34., 28., 44., 22., 43., 38., 19., 52., 37.,
       23., 68., 56., 71., 20., 61., 58., 46., 69., 65., 30., 81., 72.,
       67., 62., 18., 16., 74., 76., 80., 73., 70., 17., 15., 75., 77.,
       14., 78., 82., 83., 85.,  0., 84., 79., 91., 86., 88.])

In [9]:
targets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264325 entries, 350459 to 103148
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   age      264325 non-null  float64
 1   is_male  264325 non-null  Int32  
 2   user_id  264325 non-null  Int32  
dtypes: Int32(2), float64(1)
memory usage: 6.6 MB


In [10]:
sumbit.user_id = sumbit.user_id.astype('Int32')

In [11]:
sumbit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144724 entries, 221301 to 145315
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user_id  144724 non-null  Int32
dtypes: Int32(1)
memory usage: 1.8 MB


In [12]:
def age_bucket1(x):
    if x >= 19 and x <= 25:
        return 1
    if x >= 26 and x <= 35:
        return 2
    if x >= 36 and x <= 45:
        return 3
    if x >= 46 and x <= 55:
        return 4
    if x >= 56 and x <= 65:
        return 5
    if x >= 66:
        return 6   

In [13]:
targets.age = targets.age.map(age_bucket1)

In [14]:
targets.age = targets.age.astype('Int32')

In [15]:
targets = targets.dropna()

In [16]:
targets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263508 entries, 350459 to 103148
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   age      263508 non-null  Int32
 1   is_male  263508 non-null  Int32
 2   user_id  263508 non-null  Int32
dtypes: Int32(3)
memory usage: 5.8 MB


In [17]:
df1 = pl.read_parquet('data_url.parquet')
df2 = pl.read_parquet('data_category.parquet')

In [18]:
df = df1.join(df2, left_on='user_id', right_on='user_id')

In [19]:
df.head()

user_id,ad.adriver.ru,apple.com,avatars.mds.yandex.net,googleads.g.doubleclick.net,i.ytimg.com,icloud.com,m.avito.ru,relap.io,sun9-5.userapi.com,ads.adfox.ru,banners.adfox.ru,mail.yandex.ru,syndication.realsrv.com,ad.mail.ru,vk.com,ssp.otm-r.com,ads.betweendigital.com,s.optnx.com,yastatic.net,dsp.rtb.mts.ru,tpc.googlesyndication.com,festima.ru,sun9-85.userapi.com,s.viiadr.com,login.mts.ru,sun9-77.userapi.com,aliexpress.ru,t.me,yandex.ru,rtb.com.ru,onlinetours.ru,mamba.ru,exchange.buzzoola.com,forms.yandex.ru,sun9-80.userapi.com,rt.bongacams14.com,...,2021-12-21 00:00:00.000000000,2022-01-20 00:00:00.000000000,2022-01-22 00:00:00.000000000,2021-11-10 00:00:00.000000000,2021-12-31 00:00:00.000000000,2022-01-26 00:00:00.000000000,2021-11-09 00:00:00.000000000,2022-01-21 00:00:00.000000000,2021-12-23 00:00:00.000000000,2021-11-14 00:00:00.000000000,2022-10-15 00:00:00.000000000,2022-10-13 00:00:00.000000000,2022-10-09 00:00:00.000000000,2022-10-06 00:00:00.000000000,2022-10-14 00:00:00.000000000,2022-10-12 00:00:00.000000000,2022-10-28 00:00:00.000000000,2022-10-19 00:00:00.000000000,2022-10-07 00:00:00.000000000,2022-10-20 00:00:00.000000000,2022-10-18 00:00:00.000000000,2022-11-01 00:00:00.000000000,2022-10-27 00:00:00.000000000,2022-10-22 00:00:00.000000000,2022-10-31 00:00:00.000000000,2022-10-29 00:00:00.000000000,2022-10-21 00:00:00.000000000,2022-10-26 00:00:00.000000000,2022-10-11 00:00:00.000000000,2022-10-08 00:00:00.000000000,2022-10-16 00:00:00.000000000,2022-10-25 00:00:00.000000000,2022-10-24 00:00:00.000000000,2022-10-30 00:00:00.000000000,2022-10-17 00:00:00.000000000,2022-10-23 00:00:00.000000000,2022-10-10 00:00:00.000000000
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,...,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
45098,12,491,174,305,446,188,82,5,4,54,3,29,7,47,46,2,16,5,60,5,2,1,7,1,17,8,5,1,61,3,1,1,7,1,7,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117132,5,0,12,40,80,0,0,0,67,1,0,0,0,101,172,0,0,0,7,0,7,0,57,0,1,62,0,5,5,0,0,0,0,0,63,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79395,16,0,112,305,660,0,1,7,1,49,4,10,1,212,143,0,9,0,144,0,53,0,0,0,0,0,1,5,725,4,0,0,14,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91294,23,0,145,355,517,0,2,14,1,76,3,45,4,37,69,0,26,1,142,0,9,0,0,0,3,0,0,6,220,11,0,0,15,0,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161323,4,0,28,47,40,0,0,9,0,14,1,23,3,19,13,5,10,0,26,0,14,0,0,0,6,0,0,0,38,1,0,0,10,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
del df1, df2

In [21]:
df_train = df.join(pl.from_pandas(targets), left_on='user_id', right_on='user_id')

In [22]:
df_test = df.join(pl.from_pandas(sumbit), left_on='user_id', right_on='user_id')

In [23]:
df_train.head(1)

user_id,ad.adriver.ru,apple.com,avatars.mds.yandex.net,googleads.g.doubleclick.net,i.ytimg.com,icloud.com,m.avito.ru,relap.io,sun9-5.userapi.com,ads.adfox.ru,banners.adfox.ru,mail.yandex.ru,syndication.realsrv.com,ad.mail.ru,vk.com,ssp.otm-r.com,ads.betweendigital.com,s.optnx.com,yastatic.net,dsp.rtb.mts.ru,tpc.googlesyndication.com,festima.ru,sun9-85.userapi.com,s.viiadr.com,login.mts.ru,sun9-77.userapi.com,aliexpress.ru,t.me,yandex.ru,rtb.com.ru,onlinetours.ru,mamba.ru,exchange.buzzoola.com,forms.yandex.ru,sun9-80.userapi.com,rt.bongacams14.com,...,2022-01-22 00:00:00.000000000,2021-11-10 00:00:00.000000000,2021-12-31 00:00:00.000000000,2022-01-26 00:00:00.000000000,2021-11-09 00:00:00.000000000,2022-01-21 00:00:00.000000000,2021-12-23 00:00:00.000000000,2021-11-14 00:00:00.000000000,2022-10-15 00:00:00.000000000,2022-10-13 00:00:00.000000000,2022-10-09 00:00:00.000000000,2022-10-06 00:00:00.000000000,2022-10-14 00:00:00.000000000,2022-10-12 00:00:00.000000000,2022-10-28 00:00:00.000000000,2022-10-19 00:00:00.000000000,2022-10-07 00:00:00.000000000,2022-10-20 00:00:00.000000000,2022-10-18 00:00:00.000000000,2022-11-01 00:00:00.000000000,2022-10-27 00:00:00.000000000,2022-10-22 00:00:00.000000000,2022-10-31 00:00:00.000000000,2022-10-29 00:00:00.000000000,2022-10-21 00:00:00.000000000,2022-10-26 00:00:00.000000000,2022-10-11 00:00:00.000000000,2022-10-08 00:00:00.000000000,2022-10-16 00:00:00.000000000,2022-10-25 00:00:00.000000000,2022-10-24 00:00:00.000000000,2022-10-30 00:00:00.000000000,2022-10-17 00:00:00.000000000,2022-10-23 00:00:00.000000000,2022-10-10 00:00:00.000000000,age,is_male
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,...,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i32,i32
79395,16,0,112,305,660,0,1,7,1,49,4,10,1,212,143,0,9,0,144,0,53,0,0,0,0,0,1,5,725,4,0,0,14,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1


In [24]:
df_test.head(1)

user_id,ad.adriver.ru,apple.com,avatars.mds.yandex.net,googleads.g.doubleclick.net,i.ytimg.com,icloud.com,m.avito.ru,relap.io,sun9-5.userapi.com,ads.adfox.ru,banners.adfox.ru,mail.yandex.ru,syndication.realsrv.com,ad.mail.ru,vk.com,ssp.otm-r.com,ads.betweendigital.com,s.optnx.com,yastatic.net,dsp.rtb.mts.ru,tpc.googlesyndication.com,festima.ru,sun9-85.userapi.com,s.viiadr.com,login.mts.ru,sun9-77.userapi.com,aliexpress.ru,t.me,yandex.ru,rtb.com.ru,onlinetours.ru,mamba.ru,exchange.buzzoola.com,forms.yandex.ru,sun9-80.userapi.com,rt.bongacams14.com,...,2021-12-21 00:00:00.000000000,2022-01-20 00:00:00.000000000,2022-01-22 00:00:00.000000000,2021-11-10 00:00:00.000000000,2021-12-31 00:00:00.000000000,2022-01-26 00:00:00.000000000,2021-11-09 00:00:00.000000000,2022-01-21 00:00:00.000000000,2021-12-23 00:00:00.000000000,2021-11-14 00:00:00.000000000,2022-10-15 00:00:00.000000000,2022-10-13 00:00:00.000000000,2022-10-09 00:00:00.000000000,2022-10-06 00:00:00.000000000,2022-10-14 00:00:00.000000000,2022-10-12 00:00:00.000000000,2022-10-28 00:00:00.000000000,2022-10-19 00:00:00.000000000,2022-10-07 00:00:00.000000000,2022-10-20 00:00:00.000000000,2022-10-18 00:00:00.000000000,2022-11-01 00:00:00.000000000,2022-10-27 00:00:00.000000000,2022-10-22 00:00:00.000000000,2022-10-31 00:00:00.000000000,2022-10-29 00:00:00.000000000,2022-10-21 00:00:00.000000000,2022-10-26 00:00:00.000000000,2022-10-11 00:00:00.000000000,2022-10-08 00:00:00.000000000,2022-10-16 00:00:00.000000000,2022-10-25 00:00:00.000000000,2022-10-24 00:00:00.000000000,2022-10-30 00:00:00.000000000,2022-10-17 00:00:00.000000000,2022-10-23 00:00:00.000000000,2022-10-10 00:00:00.000000000
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,...,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
45098,12,491,174,305,446,188,82,5,4,54,3,29,7,47,46,2,16,5,60,5,2,1,7,1,17,8,5,1,61,3,1,1,7,1,7,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df_test

user_id,ad.adriver.ru,apple.com,avatars.mds.yandex.net,googleads.g.doubleclick.net,i.ytimg.com,icloud.com,m.avito.ru,relap.io,sun9-5.userapi.com,ads.adfox.ru,banners.adfox.ru,mail.yandex.ru,syndication.realsrv.com,ad.mail.ru,vk.com,ssp.otm-r.com,ads.betweendigital.com,s.optnx.com,yastatic.net,dsp.rtb.mts.ru,tpc.googlesyndication.com,festima.ru,sun9-85.userapi.com,s.viiadr.com,login.mts.ru,sun9-77.userapi.com,aliexpress.ru,t.me,yandex.ru,rtb.com.ru,onlinetours.ru,mamba.ru,exchange.buzzoola.com,forms.yandex.ru,sun9-80.userapi.com,rt.bongacams14.com,...,2021-12-21 00:00:00.000000000,2022-01-20 00:00:00.000000000,2022-01-22 00:00:00.000000000,2021-11-10 00:00:00.000000000,2021-12-31 00:00:00.000000000,2022-01-26 00:00:00.000000000,2021-11-09 00:00:00.000000000,2022-01-21 00:00:00.000000000,2021-12-23 00:00:00.000000000,2021-11-14 00:00:00.000000000,2022-10-15 00:00:00.000000000,2022-10-13 00:00:00.000000000,2022-10-09 00:00:00.000000000,2022-10-06 00:00:00.000000000,2022-10-14 00:00:00.000000000,2022-10-12 00:00:00.000000000,2022-10-28 00:00:00.000000000,2022-10-19 00:00:00.000000000,2022-10-07 00:00:00.000000000,2022-10-20 00:00:00.000000000,2022-10-18 00:00:00.000000000,2022-11-01 00:00:00.000000000,2022-10-27 00:00:00.000000000,2022-10-22 00:00:00.000000000,2022-10-31 00:00:00.000000000,2022-10-29 00:00:00.000000000,2022-10-21 00:00:00.000000000,2022-10-26 00:00:00.000000000,2022-10-11 00:00:00.000000000,2022-10-08 00:00:00.000000000,2022-10-16 00:00:00.000000000,2022-10-25 00:00:00.000000000,2022-10-24 00:00:00.000000000,2022-10-30 00:00:00.000000000,2022-10-17 00:00:00.000000000,2022-10-23 00:00:00.000000000,2022-10-10 00:00:00.000000000
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,...,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
45098,12,491,174,305,446,188,82,5,4,54,3,29,7,47,46,2,16,5,60,5,2,1,7,1,17,8,5,1,61,3,1,1,7,1,7,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117132,5,0,12,40,80,0,0,0,67,1,0,0,0,101,172,0,0,0,7,0,7,0,57,0,1,62,0,5,5,0,0,0,0,0,63,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204489,6,0,56,72,77,0,2,4,12,35,1,23,16,49,143,0,2,0,36,0,15,0,20,0,0,17,0,3,163,6,0,0,3,0,8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148545,24,0,202,814,662,0,9,7,4,104,4,31,0,202,654,0,31,0,247,1,26,0,7,0,1,9,0,8,683,6,0,0,13,0,11,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316847,4,0,145,677,736,0,0,4,0,24,4,1,0,39,324,0,12,0,57,1,8,0,4,0,6,5,0,11,209,1,0,0,8,0,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205675,4,0,33,37,22,0,0,2,3,25,1,35,0,29,29,0,20,0,29,0,26,0,2,0,3,0,0,0,29,1,0,1,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
399764,4,0,126,172,192,0,0,6,0,51,4,77,0,82,22,2,9,0,209,0,18,0,0,0,0,0,3,0,504,8,0,0,5,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
216400,8,0,138,177,121,0,18,3,2,0,1,86,0,251,87,5,46,0,117,0,45,0,3,0,0,2,0,0,171,3,0,0,8,0,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360049,11,0,129,143,74,0,1,7,18,79,1,116,0,100,253,8,14,0,1

In [26]:
df_train.write_parquet('df_train.parquet')
df_test.write_parquet('df_test.parquet')